In [ ]:
# -*- coding: utf-8 -*-
"""
Created in Jupyter Notebook

Rozwiązanie trzech zadań:
(a) Prognozowanie zużycia wody
(b) Wykrywanie anomalii w systemie klimatyzacji
(c) Rozpoznawanie wzorców przemieszczania się w biurze
"""

# =============================================================================
# IMPORT BIBLIOTEK
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, silhouette_score
from sklearn.neighbors import LocalOutlierFactor
import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

# =============================================================================
# (a) PROGNOZOWANIE ZUŻYCIA WODY
# =============================================================================
print("="*80)
print("(a) PROGNOZOWANIE ZUŻYCIA WODY W GOSPODARSTWIE DOMOWYM")
print("="*80)

# Generowanie syntetycznych danych
dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
water_usage = np.sin(np.linspace(0, 10, len(dates))) * 10 + 20 + np.random.normal(0, 2, len(dates))
water_data = pd.Series(water_usage, index=dates, name='WaterUsage')

# Wizualizacja danych
plt.figure(figsize=(14, 6))
water_data.plot(title='Dzienne zużycie wody', grid=True)
plt.ylabel('Zużycie (litry)')
plt.tight_layout()
plt.show()

# Analiza sezonowości
plt.figure(figsize=(14, 6))
pd.plotting.autocorrelation_plot(water_data)
plt.title('Autokorelacja zużycia wody')
plt.grid(True)
plt.show()

# Podział danych i model SARIMA
train = water_data.iloc[:-30]
test = water_data.iloc[-30:]

model = SARIMAX(train, 
                order=(1, 0, 1), 
                seasonal_order=(1, 1, 1, 7),
                enforce_stationarity=False,
                enforce_invertibility=False)

results = model.fit(disp=False)

# Prognoza i ewaluacja
forecast = results.get_forecast(steps=30)
predicted = forecast.predicted_mean
conf_int = forecast.conf_int()

mae = mean_absolute_error(test, predicted)
print(f'\nŚredni błąd bezwzględny (MAE): {mae:.2f} litrów')
print(f"Średnie zużycie wody: {water_data.mean():.2f} litrów")

# Wizualizacja wyników
plt.figure(figsize=(14, 7))
plt.plot(train.index, train, 'b-', label='Dane treningowe')
plt.plot(test.index, test, 'g-', label='Rzeczywiste wartości')
plt.plot(predicted.index, predicted, 'r--', label='Prognoza')
plt.fill_between(conf_int.index, 
                 conf_int.iloc[:, 0],
                 conf_int.iloc[:, 1], 
                 color='pink', alpha=0.3, label='Przedział ufności')
plt.title('Prognoza zużycia wody na 30 dni', fontsize=14)
plt.xlabel('Data')
plt.ylabel('Zużycie wody (litry)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# =============================================================================
# (b) WYKRYWANIE ANOMALII W SYSTEMIE KLIMATYZACJI
# =============================================================================
print("\n" + "="*80)
print("(b) WYKRYWANIE ANOMALII W DANYCH Z SYSTEMU KLIMATYZACJI")
print("="*80)

# Generowanie danych
dates = pd.date_range(start='2023-06-01', periods=1000, freq='H')
temperature = 22 + 5 * np.sin(np.linspace(0, 20, len(dates))) + np.random.normal(0, 0.8, len(dates))

# Dodanie anomalii
temperature[200] = 35  # Nagły wzrost
temperature[500] = 10  # Nagły spadek
temperature[700:705] = 30  # Dłuższa anomalia
temperature[900] = -5  # Ekstremalna wartość

ac_data = pd.DataFrame({
    'timestamp': dates,
    'temperature': temperature,
    'humidity': 40 + 10 * np.sin(np.linspace(0, 15, len(dates))) + np.random.normal(0, 3, len(dates))
})
ac_data.set_index('timestamp', inplace=True)

# Wizualizacja danych
fig, ax = plt.subplots(2, 1, figsize=(14, 10))
ac_data['temperature'].plot(ax=ax[0], title='Temperatura systemu klimatyzacji', grid=True)
ax[0].set_ylabel('Temperatura (°C)')
ac_data['humidity'].plot(ax=ax[1], title='Wilgotność systemu klimatyzacji', grid=True)
ax[1].set_ylabel('Wilgotność (%)')
plt.tight_layout()
plt.show()

# Wykrywanie anomalii
scaler = StandardScaler()
X = scaler.fit_transform(ac_data)

# Isolation Forest
iso_forest = IsolationForest(contamination=0.03, random_state=42)
ac_data['anomaly_if'] = iso_forest.fit_predict(X)
ac_data['anomaly_if'] = ac_data['anomaly_if'].map({1: 0, -1: 1})  # 1 = anomalia

# LOF
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.03)
ac_data['anomaly_lof'] = lof.fit_predict(X)
ac_data['anomaly_lof'] = ac_data['anomaly_lof'].map({1: 0, -1: 1})  # 1 = anomalia

# Kombinacja wyników
ac_data['anomaly'] = ac_data[['anomaly_if', 'anomaly_lof']].max(axis=1)

# Wizualizacja anomalii
plt.figure(figsize=(14, 8))
plt.plot(ac_data.index, ac_data['temperature'], 'b-', label='Temperatura')
anomalies = ac_data[ac_data['anomaly'] == 1]
plt.scatter(anomalies.index, anomalies['temperature'], 
            color='red', s=50, label='Anomalia')
plt.title('Wykryte anomalie w systemie klimatyzacji', fontsize=14)
plt.xlabel('Czas')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Statystyki anomalii
print(f"\nWykryte anomalie: {ac_data['anomaly'].sum()} ({ac_data['anomaly'].mean()*100:.1f}% danych)")
print("Przykładowe anomalie:")
print(ac_data[ac_data['anomaly'] == 1].head())

# =============================================================================
# (c) ROZPOZNAWANIE WZORCÓW PRZEMIESZCZANIA SIĘ W BIURZE
# =============================================================================
print("\n" + "="*80)
print("(c) ROZPOZNAWANIE WZORCÓW PRZEMIESZCZANIA SIĘ W BIURZE")
print("="*80)

# Generowanie danych
np.random.seed(42)
num_points = 500

# Symulacja różnych obszarów w biurze
zones = {
    'Strefa pracy': {'x': (10, 40), 'y': (60, 90), 'size': 150},
    'Strefa relaksu': {'x': (60, 90), 'y': (70, 90), 'size': 100},
    'Kuchnia': {'x': (70, 90), 'y': (10, 40), 'size': 80},
    'Sale konf': {'x': (10, 40), 'y': (10, 40), 'size': 100},
    'Recepcja': {'x': (40, 60), 'y': (40, 60), 'size': 70}
}

data = []
for zone, params in zones.items():
    for _ in range(params['size']):
        x = np.random.uniform(*params['x'])
        y = np.random.uniform(*params['y'])
        data.append([x, y, zone])
        
df = pd.DataFrame(data, columns=['x', 'y', 'zone'])
movement_data = df[['x', 'y']].sample(frac=1).reset_index(drop=True)

# Wizualizacja
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df, x='x', y='y', hue='zone', palette='viridis', s=80, alpha=0.8)
plt.title('Mapa przemieszczeń w biurze', fontsize=14)
plt.xlabel('Oś X')
plt.ylabel('Oś Y')
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.grid(True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Analiza skupień
scaler = StandardScaler()
X_scaled = scaler.fit_transform(movement_data)

# Wybór optymalnej liczby klastrów
inertia = []
silhouette = []
k_range = range(2, 10)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)
    silhouette.append(silhouette_score(X_scaled, kmeans.labels_))

# Wykres łokcia
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(k_range, inertia, 'bo-')
plt.xlabel('Liczba klastrów')
plt.ylabel('Inercja')
plt.title('Metoda łokcia')

plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette, 'go-')
plt.xlabel('Liczba klastrów')
plt.ylabel('Współczynnik sylwetowy')
plt.title('Jakość grupowania')
plt.tight_layout()
plt.show()

# Grupowanie z optymalną liczbą klastrów
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
movement_data['cluster'] = kmeans.fit_predict(X_scaled)

# Wizualizacja klastrów
plt.figure(figsize=(12, 8))
sns.scatterplot(data=movement_data, x='x', y='y', hue='cluster', 
                palette='viridis', s=80, alpha=0.8)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            s=300, c='red', marker='X', label='Centroidy')
plt.title(f'Grupowanie przemieszczeń (k={n_clusters})', fontsize=14)
plt.xlabel('Oś X')
plt.ylabel('Oś Y')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# Analiza klastrów
print("\nAnaliza klastrów:")
print(f"Liczba punktów w klastrach:")
print(movement_data['cluster'].value_counts())

# Charakterystyka klastrów
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
print("\nCentroidy klastrów:")
for i, center in enumerate(cluster_centers):
    print(f"Klaster {i}: ({center[0]:.1f}, {center[1]:.1f})")